In [1]:
import os, sys
print("python =", sys.executable)
print("PASS =", bool(os.getenv("AIDEV_DB_PASS")))
print("DATA =", os.getenv("AIDEV_DATA_DIR"))


python = /Users/young/MSR/script/.venv/bin/python
PASS = False
DATA = None


In [1]:
import os
import time
from pathlib import Path

from dotenv import load_dotenv
load_dotenv()


import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# ---- config (env-first) ----
DB_USER = os.getenv("AIDEV_DB_USER", "root")
DB_PASS = os.getenv("AIDEV_DB_PASS")          # required
DB_HOST = os.getenv("AIDEV_DB_HOST", "127.0.0.1")
DB_PORT = int(os.getenv("AIDEV_DB_PORT", "3306"))
DB_NAME = os.getenv("AIDEV_DB_NAME", "aidev")

DATA_DIR = Path(os.getenv("AIDEV_DATA_DIR", "data")).expanduser().resolve()
IF_EXISTS = os.getenv("AIDEV_IF_EXISTS", "replace")  # replace/append/fail

if not DB_PASS:
    raise ValueError("Missing AIDEV_DB_PASS")

# ---- db engine ----
password = quote_plus(DB_PASS)  # URL-escape
engine = create_engine(
    f"mysql+pymysql://{DB_USER}:{password}@{DB_HOST}:{DB_PORT}/{DB_NAME}?charset=utf8mb4",
    pool_pre_ping=True,
)

with engine.connect() as conn:
    print("DB test:", conn.exec_driver_sql("SELECT 1").fetchone())

print("DATA_DIR:", DATA_DIR)
print("IF_EXISTS:", IF_EXISTS)

# ---- inputs ----
tables = [
    "pull_request",
    "pr_comments",
    "pr_reviews",
    "pr_review_comments_v2",
    "pr_commits",
    "pr_timeline",
    "user",
]
name_map = {"user": "aidev_user"}  # reserved name
DROP_COLS = {
    "body", "diff", "patch", "description", "message", "text", "content",
    "diff_hunk", "hunk", "raw_diff", "raw_patch", "code", "snippet"
}

# ---- precheck ----
missing = [str((DATA_DIR / f"{t}.parquet")) for t in tables if not (DATA_DIR / f"{t}.parquet").exists()]
if missing:
    raise FileNotFoundError("Missing parquet:\n" + "\n".join(missing))

# ---- import loop ----
for t in tables:
    t0 = time.time()

    path = DATA_DIR / f"{t}.parquet"
    df = pd.read_parquet(path)
    df = df.drop(columns=[c for c in df.columns if c in DROP_COLS], errors="ignore")  # shrink

    target = name_map.get(t, t)

    print(f"\n=== {t} -> {target} ===")
    print("shape:", df.shape)

    df.to_sql(
        target,
        con=engine,
        if_exists=IF_EXISTS,
        index=False,
        chunksize=50_000,
        method="multi",
    )

    print(f"done: {time.time() - t0:.1f}s")

engine.dispose()  # release
print("Import done.")


DB test: (1,)
DATA_DIR: /Users/young/Downloads/MDS/M3/542 data wrangling/data
IF_EXISTS: replace

=== pull_request -> pull_request ===
shape: (33596, 13)
done: 9.4s

=== pr_comments -> pr_comments ===
shape: (39122, 6)
done: 3.0s

=== pr_reviews -> pr_reviews ===
shape: (28875, 6)
done: 2.3s

=== pr_review_comments_v2 -> pr_review_comments_v2 ===
shape: (26868, 13)
done: 4.7s

=== pr_commits -> pr_commits ===
shape: (88576, 4)
done: 5.0s

=== pr_timeline -> pr_timeline ===
shape: (325500, 7)
done: 28.0s

=== user -> aidev_user ===
shape: (1796, 5)
done: 0.1s
Import done.


In [2]:
from sqlalchemy import text

tables = [
    "pull_request",
    "pr_comments",
    "pr_reviews",
    "pr_review_comments_v2",
    "pr_commits",
    "pr_timeline",
    "aidev_user",
]

with engine.connect() as conn:
    for t in tables:
        n = conn.execute(text(f"SELECT COUNT(*) FROM {t}")).scalar()
        print(t, n)


pull_request 33596
pr_comments 39122
pr_reviews 28875
pr_review_comments_v2 26868
pr_commits 88576
pr_timeline 325500
aidev_user 1796


In [7]:
from pathlib import Path
import pandas as pd

OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True)

JOBS = [
    ("pr_scenarios_rq1.sql", "pr_scenarios_rq1", "rq1_pr_scenarios"),
    ("pr_cost_rq2.sql", "pr_cost_rq2", "rq2_pr_cost"),
    ("RQ23.sql", "pr_rq12", "rq23"),
]

def read_sql_file(fp: str) -> str:
    p = Path(fp)
    if not p.exists():
        raise FileNotFoundError(f"Missing SQL file: {p.resolve()}")
    return p.read_text(encoding="utf-8")

def split_sql(sql_text: str):
    stmts = []
    for s in sql_text.split(";"):
        s = s.strip()
        if not s:
            continue
        lines = []
        for line in s.splitlines():
            t = line.strip()
            if t.startswith("--"):
                continue
            lines.append(line)
        stmt = "\n".join(lines).strip()
        if stmt:
            stmts.append(stmt)
    return stmts

def should_skip(stmt: str) -> bool:
    s = stmt.strip().lower()
    return s.startswith("create database") or s.startswith("use ")

def exec_sql_allow_duplicates(conn, stmt: str):
    try:
        conn.exec_driver_sql(stmt)
    except Exception as e:
        msg = str(e)
        # MySQL 1061: Duplicate key name (index exists)
        if "1061" in msg and "Duplicate key name" in msg:
            return
        raise

with engine.begin() as conn:
    for sql_file, obj_name, out_stem in JOBS:
        sql_text = read_sql_file(sql_file)

        for stmt in split_sql(sql_text):
            if should_skip(stmt):
                continue
            exec_sql_allow_duplicates(conn, stmt)

        df = pd.read_sql_query(f"SELECT * FROM {obj_name}", conn)

        out_path = OUT_DIR / f"{out_stem}.csv"
        df.to_csv(out_path, index=False)
        print(out_stem, "shape=", df.shape, "saved=", out_path.resolve())


rq1_pr_scenarios shape= (33596, 12) saved= /Users/young/MSR/script/outputs/rq1_pr_scenarios.csv
rq2_pr_cost shape= (33596, 5) saved= /Users/young/MSR/script/outputs/rq2_pr_cost.csv
rq23 shape= (33596, 16) saved= /Users/young/MSR/script/outputs/rq23.csv
